# Comparison of model-based and MRI-based target volume definition for ocular proton therapy 
These scripts provide methods to generate polynomial tumour models, as commonly used in ocular proton therapy planning, based on 3D tumour delineations. Furthermore, it provides methods to compare the resulting model to the tumour delineations. 

More figures and details about the methods are available within the functions.

L. Klaassen, 2025. 


In [1]:
import os

import alphashape
import manifold3d
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymeshfix
import scipy
import shapely
import sklearn
import trimesh
from Automatic_measurements import calc_LBD, calc_Prom_Centre
from Generate_tumour_model import (
    directed_angle_between_vectors,
    generate_tumour_model,
    generate_tumour_model_extrathickness,
    save_point_cloud_as_stl,
    upsample_point_cloud,
)
from Model_evaluation import signed_surface_dist, volume_analysis
from Prepare_base import correct_base, expand_base, redefine_prom
from tqdm.auto import tqdm

### Preparations

NOTE: you should specify the path to the STL files below.

In [ ]:
# Define paths
filedir = r'' #Enter the path where STL files are
outdir = r'' # Enter the path where you want the models to go

tumour_path = os.path.join(filedir, 'Tumour_example.stl')
eye_path = os.path.join(filedir, 'Eye_example.stl')

if not os.path.isdir(outdir):
    os.makedirs(outdir)


In [3]:
#Load stl
tumour = trimesh.load(tumour_path)
eye = trimesh.load(eye_path)

# Tumour measurements
thickness, thickness_base, thickness_top = calc_Prom_Centre(tumour,eye, include_sclera = False) # Prominence = thickness
lbd, lbd_coor1, lbd_coor2, base = calc_LBD(tumour,eye)
print('Thickness is ', round(thickness,1), 'mm, largest diameter is ', round(lbd,1), 'mm')

# Define tumour base and expanded tumour base
corrected_base, corrected_base_normals = correct_base(tumour,eye, threshold_angle = 45)
expanded_base, new_points = expand_base(corrected_base, max_distance = 1.0)

# Upsample point clouds for tumour base
corrected_base, new_points2 = upsample_point_cloud(corrected_base, factor=10, k=10) # Upsample base by factor 10 with 10 nearest neighbours

# Check if thickness needs to be corrected (automatic thickness determination is less accurate in tumours with middle of the eye within or very close to tumour)
thickness_base, redefined = redefine_prom(tumour,eye, corrected_base, thickness_base)
        

Thickness is  9.1 mm, largest diameter is  17.7 mm


### Construct tumour models

In [4]:
# Construct tumour models: standard version without expansions
for sf in tqdm(np.arange(1,10,1)): #make models for shape factors 1-10 in steps of 1
    try:
        sf = round(sf,1)
        
        #location of files
        out = os.path.join(outdir, 'Testpatient_tumourmodel_sf{0}.stl'.format(sf))

        tumour_model_points = generate_tumour_model(tumour,eye, corrected_base, sf) #generate tumour model
        save_point_cloud_as_stl(tumour_model_points, out, max_attempts=5) #save tumour model as STL
        
        # Uncomment this section for a figure of the delineation and tumour model created
        #fig = plt.figure()
        #axes = fig.add_subplot(projection='3d')
        #axes.plot(eye.vertices[:,0], eye.vertices[:,1], eye.vertices[:,2], c='w', alpha = 0.8)
        #axes.scatter(tumour.vertices[:,0], tumour.vertices[:,1], tumour.vertices[:,2], c='g', alpha = 0.4, label = 'delineated tumour')
        #axes.scatter(tumour_model_points[:,0], tumour_model_points[:,1], tumour_model_points[:,2], c= 'b', alpha = 0.7, label = 'tumour model')

        #axes.axis('equal')
        #axes.set_title('Tumour model and tumour delineation SF {0}'.format(sf))
        #fig.legend()

    except Exception as e: print(e)


  0%|          | 0/9 [00:00<?, ?it/s]

   Saved STL file: /tmp/Testpatient_tumourmodel_sf1.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Mesh not a volume, upsampling points (Attempt 2)...
   Saved STL file: /tmp/Testpatient_tumourmodel_sf2.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Mesh not a volume, upsampling points (Attempt 2)...
   Saved STL file: /tmp/Testpatient_tumourmodel_sf3.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Mesh not a volume, upsampling points (Attempt 2)...
   Saved STL file: /tmp/Testpatient_tumourmodel_sf4.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Saved STL file: /tmp/Testpatient_tumourmodel_sf5.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Mesh not a volume, upsampling points (Attempt 2)...
   Mesh not a volume, upsampling points (Attempt 3)...
   Saved STL file: /tmp/Testpatient_tumourmodel_sf6.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_sf7.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   

In [5]:
# Construct tumour models: version with extra thickness
for sf in tqdm(np.arange(1,10,1)): #Make models for shape factors 1-10 in steps of 1
    try:
        sf = round(sf,1)

        #location of files
        out = os.path.join(outdir, 'Testpatient_tumourmodel_extrathickness_sf{0}.stl'.format(sf))

        tumour = trimesh.load(tumour_path)
        eye = trimesh.load(eye_path)

        tumour_model_points_extrathickness = generate_tumour_model_extrathickness(tumour,eye, corrected_base, sf, addedthickness = 0.5) #generate tumour model with 0.5 mm added thickness
        save_point_cloud_as_stl(tumour_model_points_extrathickness, out, max_attempts=5) #save tumour model as STL

        # Uncomment this section for a figure of the delineation and tumour model created
        #fig = plt.figure()
        #axes = fig.add_subplot(projection='3d')
        #axes.plot(eye.vertices[:,0], eye.vertices[:,1], eye.vertices[:,2], c='w', alpha = 0.8)
        #axes.scatter(tumour.vertices[:,0], tumour.vertices[:,1], tumour.vertices[:,2], c='g', alpha = 0.4, label = 'delineated tumour')
        #axes.scatter(tumour_model_points_extrathickness[:,0], tumour_model_points_extrathickness[:,1], tumour_model_points_extrathickness[:,2], c= 'b', alpha = 0.7, label = 'tumour model')

        #axes.axis('equal')
        #axes.set_title('Tumour model and tumour delineation SF {0}'.format(sf))
        #fig.legend()

    except Exception as e: print(e)
                

  0%|          | 0/9 [00:00<?, ?it/s]

   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf1.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Mesh not a volume, upsampling points (Attempt 2)...
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf2.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf3.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf4.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Mesh not a volume, upsampling points (Attempt 2)...
   Mesh not a volume, upsampling points (Attempt 3)...
   Mesh not a volume, upsampling points (Attempt 4)...
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf5.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf6.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf7.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf8.stl
   Mesh not a volume, upsampling points (Attemp

In [6]:
# Construct tumour models: version with expanded base
for sf in tqdm(np.arange(1,10,1)): #make models for shape factors 1-10 in steps of 1
    try:
        sf = round(sf,1)

        #location of files
        out = os.path.join(outdir, 'Testpatient_tumourmodel_extrabase_sf{0}.stl'.format(sf))

        tumour = trimesh.load(tumour_path)
        eye = trimesh.load(eye_path)

        tumour_model_points_extrabase = generate_tumour_model(tumour,eye, expanded_base, sf) #generate tumour model 
        save_point_cloud_as_stl(tumour_model_points_extrabase, out, max_attempts=5) #save tumour model as STL

        # Uncomment this section for a figure of the delineation and tumour model created
        #fig = plt.figure()
        #axes = fig.add_subplot(projection='3d')
        #axes.plot(eye.vertices[:,0], eye.vertices[:,1], eye.vertices[:,2], c='w', alpha = 0.8)
        #axes.scatter(tumour.vertices[:,0], tumour.vertices[:,1], tumour.vertices[:,2], c='g', alpha = 0.4, label = 'delineated tumour')
        #axes.scatter(tumour_model_points_extrabase[:,0], tumour_model_points_extrabase[:,1], tumour_model_points_extrabase[:,2], c= 'b', alpha = 0.7, label = 'tumour model')

        #axes.axis('equal')
        #axes.set_title('Tumour model and tumour delineation SF {0}'.format(sf))
        #fig.legend()

    except Exception as e: print(e)


  0%|          | 0/9 [00:00<?, ?it/s]

   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf1.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf2.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf3.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf4.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf5.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf6.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf7.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf8.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf9.stl


In [7]:
# Construct tumour models: version with added thickness and expanded base
for sf in tqdm(np.arange(1,10,1)): #make models for shape factors 1-10 in steps of 1
    try:
        sf = round(sf,1)
        
        #location of files
        out = os.path.join(outdir, 'Testpatient_tumourmodel_extrathickness_extrabase_sf{0}.stl'.format(sf))
        
        tumour = trimesh.load(tumour_path)
        eye = trimesh.load(eye_path)

        tumour_model_points_extrathickness_extrabase = generate_tumour_model_extrathickness(tumour,eye, expanded_base, sf, addedthickness = 0.5) #generate tumour model with 0.5 mm added thickness
        save_point_cloud_as_stl(tumour_model_points_extrathickness_extrabase, out, max_attempts=5) #save tumour model as STL
        
        # Uncomment this section for a figure of the delineation and tumour model created
        #fig = plt.figure()
        #axes = fig.add_subplot(projection='3d')
        #axes.plot(eye.vertices[:,0], eye.vertices[:,1], eye.vertices[:,2], c='w', alpha = 0.8)
        #axes.scatter(tumour.vertices[:,0], tumour.vertices[:,1], tumour.vertices[:,2], c='g', alpha = 0.4, label = 'delineated tumour')
        #axes.scatter(tumour_model_points_extrathickness_extrabase[:,0], tumour_model_points_extrathickness_extrabase[:,1], tumour_model_points_extrathickness_extrabase[:,2], c= 'b', alpha = 0.7, label = 'tumour model')

        #axes.axis('equal')
        #axes.set_title('Tumour model and tumour delineation SF {0}'.format(sf))
        #fig.legend()

    except Exception as e: print(e)

  0%|          | 0/9 [00:00<?, ?it/s]

   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf1.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf2.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf3.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf4.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf5.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf6.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf7.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf8.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf9.stl


### Evaluate tumour models 
If tumour models have unexpected shapes, sometimes it is sufficient to run the model generation again for that shape factor.

In [8]:
# Evaluation for standard model
volume_metrics_perpatient = {}
distance_metrics_perpatient = {}

for sf in tqdm(np.arange(1,10,1)):
    try:
        sf = round(sf,1)
        tqdm.write('   started with sf {0}'.format(sf))
        model_path = os.path.join(outdir, 'Testpatient_tumourmodel_sf{0}.stl'.format(sf)) 
        model = trimesh.load(model_path)
        eye = trimesh.load(eye_path)

        volume_metrics = volume_analysis(tumour, model)
        volume_metrics_perpatient[r'SF{0}'.format(sf)] = volume_metrics

        dists, dist_metrics = signed_surface_dist(tumour, model, eye)
        distance_metrics_perpatient[r'SF{0}'.format(sf)] = dist_metrics
        #print('    ', dist_metrics)
    except Exception as e:
        print(e)
        volume_metrics_perpatient[r'SF{0}'.format(sf)] = {'overlap_abs': np.nan, 'overlap_rel': np.nan, 'overestimation_abs': np.nan,
                'overestimation_rel': np.nan, 'underestimation_abs': np.nan,
                'underestimation_rel': np.nan, 'IoU': np.nan}
        distance_metrics_perpatient[r'SF{0}'.format(sf)] = {'surf_dist_median_abs':  np.nan, 'surf_dist_min': np.nan, 'surf_dist_perc_0.5': np.nan,'surf_dist_perc_1': np.nan,'surf_dist_perc_2': np.nan,
                                                            'surf_dist_perc_5':  np.nan, 'surf_dist_perc_25':  np.nan, 
                'surf_dist_perc_50':  np.nan, 'surf_dist_perc_75':  np.nan, 'surf_dist_perc_95':  np.nan, 'surf_dist_max': np.nan}


  0%|          | 0/9 [00:00<?, ?it/s]

   started with sf 1
   started with sf 2
   started with sf 3
   started with sf 4
   started with sf 5
   started with sf 6
   started with sf 7
   started with sf 8
   started with sf 9


In [9]:
# Put the results in a dataframe
rows = []
rows2 = []

for sf, metrics in volume_metrics_perpatient.items():
    sf_number = float(sf.replace('SF', ''))
    rows.append({
            'SF': sf_number,
            'overlap_abs': metrics['overlap_abs'],
            'overlap_rel': metrics['overlap_rel'],
            'overestimation_abs': metrics['overestimation_abs'],
            'overestimation_rel': metrics['overestimation_rel'],
            'underestimation_abs': metrics['underestimation_abs'],
            'underestimation_rel': metrics['underestimation_rel'],
            'IoU': metrics['IoU'],
        })


for sf, metrics in distance_metrics_perpatient.items():
    sf_number = float(sf.replace('SF', ''))
    rows2.append({
            'SF': sf_number,
            'surf_dist_median_abs': metrics['surf_dist_median_abs'],
            'surf_dist_min': metrics['surf_dist_min'],
            'surf_dist_perc_0.5': metrics['surf_dist_perc_0.5'],
            'surf_dist_perc_1': metrics['surf_dist_perc_1'],
            'surf_dist_perc_2': metrics['surf_dist_perc_2'],
            'surf_dist_perc_5': metrics['surf_dist_perc_5'],
            'surf_dist_perc_25': metrics['surf_dist_perc_25'],
            'surf_dist_perc_50': metrics['surf_dist_perc_50'],
            'surf_dist_perc_75': metrics['surf_dist_perc_75'],
            'surf_dist_perc_95': metrics['surf_dist_perc_95']
        })

volume_df = pd.DataFrame(rows)
distance_df = pd.DataFrame(rows2)

allmetrics_df = pd.DataFrame.merge(volume_df, distance_df, how = 'left')


In [10]:
# Export dataframe
out_excel_standardmodel = os.path.join(outdir, 'Evaluation_standardmodel.xlsx')
allmetrics_df.to_excel(out_excel_standardmodel)
allmetrics_df

,SF,overlap_abs,overlap_rel,overestimation_abs,overestimation_rel,underestimation_abs,underestimation_rel,IoU,surf_dist_median_abs,surf_dist_min,surf_dist_perc_0.5,surf_dist_perc_1,surf_dist_perc_2,surf_dist_perc_5,surf_dist_perc_25,surf_dist_perc_50,surf_dist_perc_75,surf_dist_perc_95
0,1.0,552.651117,50.950331,0.243619,0.022460,532.034899,49.049669,0.509389,1.522294,-2.344687,-2.270689,-2.233835,-2.163898,-2.053927,-1.338130,0.767475,1.683267,3.432138
1,2.0,1037.800301,95.677484,26.531422,2.446000,46.885719,4.322515,0.933931,0.250771,-0.791123,-0.658264,-0.606627,-0.502533,-0.383329,-0.133245,0.176876,0.324269,0.488024
2,3.0,1072.598854,98.885653,186.338632,17.179039,12.087166,1.114347,0.843885,0.665898,-0.510050,-0.431645,-0.373996,-0.261620,0.083789,0.358327,0.665898,0.938617,1.316465
3,4.0,1069.030937,98.556717,303.918578,28.019037,15.655081,1.443282,0.769860,1.040889,-0.397461,-0.277343,-0.236607,-0.128059,0.147539,0.549386,1.040889,1.450165,1.876630
4,5.0,1071.970439,98.827717,387.803736,35.752626,12.715597,1.172284,0.727999,1.221628,-0.370950,-0.227941,-0.181122,0.071207,0.172491,0.625481,1.221628,1.773865,2.298374
5,6.0,1065.075033,98.192012,444.794252,41.006728,19.610993,1.807988,0.696364,1.366895,-0.361703,-0.194079,-0.133229,0.080129,0.179441,0.710992,1.366895,2.000435,2.609518
6,7.0,1076.387325,99.234922,494.690708,45.606811,8.298701,0.765079,0.681527,1.461739,-0.364965,-0.206677,-0.137658,0.100884,0.203220,0.760434,1.461739,2.167511,2.890613
7,8.0,1057.821613,97.523301,529.106744,48.779714,26.864405,2.476699,0.655488,1.524958,-0.386186,-0.189656,-0.130260,0.098122,0.200155,0.782860,1.524958,2.255252,3.101282
8,9.0,1078.117761,99.394455,558.638144,51.502290,6.568263,0.605545,0.656059,1.576721,-0.320410,-0.168743,-0.097827,0.106758,0.207638,0.835682,1.576721,2.376236,3.324070
